In [1]:
import math
import numpy as np
import torch
from tqdm.notebook import tqdm

from liberate import fhe
from liberate.fhe import presets
import liberate

In [2]:
%%time

params = presets.params["silver"]
engine = fhe.ckks_engine(verbose=True, **params)
print(">>\tGenerate CKKS engine\n")

[2023-11-20 02:03:01.832507] I have received the context:


I have received inputs:
        buffer_bit_length		= 62
        scale_bits			= 40
        logN				= 15
        N				= 32,768
        Number of special primes	= 2
        Number of scales		= 16
        Cache folder			= '/home/hanyul/.pyenv/versions/liberate/lib/python3.11/site-packages/liberate/fhe/cache/resources'
        Security bits			= 128
        Quantum security model		= post_quantum
        Security sampling distribution	= uniform
        Number of message bits		= 60
        In total I will be using '821' bits out of available maximum '829' bits.
        And is it secured?		= True
My RNS primes are [1099510054913, 1099515691009, 1099507695617, 1099516280833, 1099506515969, 1099520606209, 1099504549889, 1099523555329, 1099503894529, 1099527946241, 1099503370241, 1099529060353, 1099498258433, 1099531223041, 1099469684737, 1099532009473, 1152921504606584833, 1152921504598720513, 1152921504597016577].
[2023-11-20 02:03:01.8

In [3]:
%%time

print("====="*20)
num_of_parties = 5
print(f"|\tnum_of_parties\t:\t{num_of_parties}")
print("-----"*20)
for level in range(engine.num_levels):

    delta = 5
    amin, amax = -255, 255
    m = engine.example(amin=amin, amax=amax)
    ############################
    ####    generate sks    ####
    ############################
    sks = [engine.create_secret_key() for _ in range(num_of_parties)]

    ############################
    ####    generate pks    ####
    ############################
    pks = [engine.create_public_key(sk=sks[0])]
    crs = engine.clone(pks[0]).data[1]
    for sk in sks[1:]:
        pks.append(engine.multiparty_create_public_key(sk, a=crs))

    cpk = engine.multiparty_create_collective_public_key(pks=pks)

    ###################
    ####    enc    ####
    ###################
    ct = engine.encorypt(m, cpk, level=level)

    ###################
    ####    dec    ####
    ###################
    pcts = [engine.multiparty_decrypt_head(ct, sks[0])]
    for sk in sks[1:]:
        pcts.append(engine.multiparty_decrypt_partial(ct, sk))
    m_ = engine.multiparty_decrypt_fusion(pcts, level=level)

    print(f"|\t>> level : {level:2d}\t{engine.absmax_error(m_, m):.15e}\t|")
    print("====="*20)

|	num_of_parties	:	5
----------------------------------------------------------------------------------------------------
|	>> level :  0	2.547722033341415e-10+9.599448347330508e-09j	|
|	>> level :  1	2.101305796031738e-10+9.494556252320763e-09j	|
|	>> level :  2	2.354454409214668e-10+9.415256130296257e-09j	|
|	>> level :  3	2.594617853901582e-10+9.473609452470555e-09j	|
|	>> level :  4	2.623892214614898e-10+9.480544349571574e-09j	|
|	>> level :  5	2.331130843913343e-10+9.515161991657806e-09j	|
|	>> level :  6	2.282902755723626e-10+9.453273719373101e-09j	|
|	>> level :  7	2.281979050167138e-10+9.418428703611426e-09j	|
|	>> level :  8	2.355307060497580e-10+9.458005933993263e-09j	|
|	>> level :  9	2.190176928706933e-10+9.469051320820654e-09j	|
|	>> level : 10	2.359001882723533e-10+9.589333771486963e-09j	|
|	>> level : 11	2.620694772303978e-10+9.447646220905881e-09j	|
|	>> level : 12	2.345359462196939e-10+9.484324436925817e-09j	|
|	>> level : 13	2.331432824576041e-10+9.474071305248799e-09